In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor

# Update the protected namespace configuration
EntityExtractor.model_config['protected_namespaces'] = ()


In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(
    model='llama3.2:latest',
    base_url='http://localhost:11434',
    # temperature=0.1,
    mean_resizing=False,
)

In [ ]:
import torch
from span_marker import SpanMarkerModel

# Load the pretrained SpanMarker model
model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")

# Move the model to GPU
if torch.cuda.is_available():
    model = model.cuda()  # Moves the model to GPU
    print("Model moved to GPU for faster predictions.")
else:
    print("CUDA is not available. Model will run on CPU.")


In [ ]:
entity_extractor = EntityExtractor(
    prediction_threshold=0.5,
    label_entities=False,  # include the entity label in the metadata (can be erroneous)
    device="cuda",  # set to "cuda" if you have a GPU or "cpu" if you have CPU alone
    model_name="tomaarsen/span-marker-mbert-base-multinerd",  # Local model name
    entity_types=["PERSON", "ORG", "GPE"],
)

In [ ]:
transformations = [
    SentenceSplitter(),
    TitleExtractor(nodes=5),  # Extracts up to 5 titles.
    QuestionsAnsweredExtractor(questions=3),  # Extracts up to 3 questions.
    SummaryExtractor(summaries=["prev", "self"]),  # Summarizes previous and current sections.
    KeywordExtractor(keywords=10),  # Extracts up to 10 keywords.
    entity_extractor,  # Detects entities with a confidence threshold of 0.5.
]


In [ ]:
%pip install nest_asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)


In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()


In [ ]:
nodes = pipeline.run(documents=documents)


In [ ]:
nodes[2].metadata

In [ ]:
for node in nodes:
    print(f"Node ID: {node.node_id}")
    print("-------------------------------------------------------------")
    print(f"Document Title: {node.metadata.get('document_title', 'N/A')}")
    print("-------------------------------------------------------------")
    print(f"Questions Answered: {node.metadata.get('questions_this_excerpt_can_answer', 'N/A')}")
    print("-------------------------------------------------------------")
    print(f"Section Summary: {node.metadata.get('section_summary', 'N/A')}")
    print("-------------------------------------------------------------")
    print(f"Excerpt Keywords: {node.metadata.get('excerpt_keywords', 'N/A')}")
    print("-------------------------------------------------------------")
    print(f"Entities: {node.metadata.get('entities', 'N/A')}")
    print("======================================================")
    print("======================================================")


In [ ]:
from llama_index.core.extractors import BaseExtractor

class CustomExtractor(BaseExtractor):
    async def aextract(self, nodes) -> list:
        metadata_list = [
            {
                "custom_metadata": node.metadata.get("document_title", "N/A") +
                "\nKeywords: " + node.metadata.get("excerpt_keywords", "N/A")
            }
            for node in nodes
        ]
        return metadata_list


In [ ]:
custom_extractor = CustomExtractor()

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        custom_extractor,
    ]
)
